Messing around with polaras rather than pandas. 

The information that I got here came from this article: https://realpython.com/polars-python/ 

High level, polars is a very performant library for doing large scale data manipulation. The key features of it are: 
* You do not need to read all data into memory 
* The computations are optimized for efficiency on your computer 
* You can do lazy evaluation which allows you to execute the steps you need only when you need it. You can create plans for how you are going to process your data before you have to process it. 
* You can integrate this into whatever pandas and numpy workflows you have since the designers of this code wanted them to be compatible. 

In [2]:
import numpy as np 
import pandas as pd 
import polars as pl

num_rows = 50000 
rng = np.random.default_rng(seed=7)

In [3]:
buildings_data = {
    'sqft': rng.exponential(scale=1000, size=num_rows), 
    'year': rng.integers(low=1995, high=2023, size=num_rows), 
    "building_type": rng.choice(['A', 'B', 'C'], size=num_rows)
}

buildings = pl.DataFrame(buildings_data)

In [4]:
buildings

sqft,year,building_type
f64,i64,str
707.529256,2017,"""B"""
1025.203348,1999,"""C"""
568.548657,1998,"""B"""
895.109864,2001,"""C"""
206.532754,1996,"""B"""
…,…,…
1528.844126,2011,"""A"""
813.274714,2020,"""C"""
1145.045726,1995,"""C"""


In [5]:
buildings.schema

Schema([('sqft', Float64), ('year', Int64), ('building_type', String)])

In [6]:
buildings.head()

sqft,year,building_type
f64,i64,str
707.529256,2017,"""B"""
1025.203348,1999,"""C"""
568.548657,1998,"""B"""
895.109864,2001,"""C"""
206.532754,1996,"""B"""


In [9]:
description = buildings.describe()

In [22]:
# You can easily select a dataframe column here like this
buildings.select(pl.col("sqft"))

# You can also do this 
buildings.select('sqft')

# If you want to filter this is easy as well 
buildings.filter(pl.col("sqft") > 700)

# another filter which might be interesting 
buildings.filter(pl.col("year") == 2008)

sqft,year,building_type
f64,i64,str
144.946517,2008,"""C"""
884.278757,2008,"""C"""
351.140355,2008,"""B"""
243.942508,2008,"""C"""
173.882447,2008,"""C"""
…,…,…
254.107102,2008,"""A"""
134.505253,2008,"""C"""
138.852877,2008,"""C"""


In [27]:
# We can do some grouping as well on this data 
buildings.group_by("building_type").agg(
    [
        pl.mean("sqft").alias("mean_sqft"), 
        pl.median("year").alias("median_year"), 
        pl.len()
    ]
)

building_type,mean_sqft,median_year,len
str,f64,f64,u32
"""A""",1009.429004,2008.0,16626
"""B""",1005.147182,2009.0,16654
"""C""",995.998607,2009.0,16720


### Working with LazyFrames

What is interesting about polars is that you can set up dataframes to do lazy execution. This means that you are only doing the evaluation of the dataframe when you need to. 

In [28]:
new_buildings = {
    'sqft': rng.exponential(scale=1000, size=num_rows), 
    'price': rng.exponential(scale=100_000, size=num_rows), 
    'year': rng.integers(low=1995, high=2023, size=num_rows), 
    'building_type': rng.choice(['A', 'B', 'C'], size=num_rows)
}

lazy_buildings = pl.LazyFrame(new_buildings)
lazy_buildings

In [30]:
lazy_query = (
    lazy_buildings
    .with_columns(
        (pl.col("price") / pl.col('sqft')).alias("price_per_sqft")
    )
    .filter(pl.col("price_per_sqft") > 100)
    .filter(pl.col("year") < 2010)
)

In [33]:
print(lazy_query.explain()) 

FILTER [(col("price_per_sqft")) > (100.0)] FROM
   WITH_COLUMNS:
   [[(col("price")) / (col("sqft"))].alias("price_per_sqft")] 
    DF ["sqft", "price", "year", "building_type"]; PROJECT */4 COLUMNS; SELECTION: [(col("year")) < (2010)]


In [34]:
lazy_query.collect().select(pl.col(['price_per_sqft', 'year']))

price_per_sqft,year
f64,i64
6116.148294,1998
234.476331,2004
305.891064,2001
1132.274686,2009
371.636692,1995
…,…
282.702528,2000
276.831319,1998
1214.328728,1996


An example that showcases how powerful the lazy api is. A lot of times, you are going to be processing data that is stored in some database somewhere or something. Bringing all of this into memory is not super efficient. We can specify different lazy queries and what not so that we can do our processing only when we need to. 

This next example comes from real python where we use some actual data 

In [39]:
import requests 
import pathlib as path

def download_file(file_url: str, local_file_path: path.Path) -> None: 
    
    response = requests.get(file_url)
    if response: 
        local_file_path.write_bytes(response.content)
        print(f'The File was successfully downloaded and stored at {local_file_path}')
    else: 
        raise requests.exceptions.RequestException(
            f'Failed to download the file. Status code: {response.status_code}'
        )

In [40]:
url = "https://data.wa.gov/api/views/f6w7-q2d2/rows.csv?accessType=DOWNLOAD"
local_file_path = path.Path("electric_cars.csv")

In [41]:
download_file(file_url=url, local_file_path=local_file_path)

The File was successfully downloaded and stored at electric_cars.csv


In [42]:
lazy_car_data = pl.scan_csv("electric_cars.csv")
lazy_car_data

In [43]:

lazy_car_data.schema

/tmp/ipykernel_7499/755857849.py:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  lazy_car_data.schema


Schema([('VIN (1-10)', String),
        ('County', String),
        ('City', String),
        ('State', String),
        ('Postal Code', Int64),
        ('Model Year', Int64),
        ('Make', String),
        ('Model', String),
        ('Electric Vehicle Type', String),
        ('Clean Alternative Fuel Vehicle (CAFV) Eligibility', String),
        ('Electric Range', Int64),
        ('Base MSRP', Int64),
        ('Legislative District', Int64),
        ('DOL Vehicle ID', Int64),
        ('Vehicle Location', String),
        ('Electric Utility', String),
        ('2020 Census Tract', Int64)])

In [45]:
lazy_car_query = (
    lazy_car_data
    .filter(pl.col("Model Year") >= 2018)
    .filter(pl.col("Electric Vehicle Type") == "Battery Electric Vehicle (BEV)")
    .group_by(['State', 'Make'])
    .agg(
        pl.mean("Electric Range").alias("Average Electric Range"), 
        pl.min("Model Year").alias("Oldest Year"), 
        pl.len().alias("Number of Cars")
    )
    .filter(pl.col("Average Electric Range") > 0)
    .filter(pl.col("Number of Cars") > 5)
    .sort(pl.col("Number of Cars"), descending=True)
)

In [46]:
lazy_car_query.collect() 

State,Make,Average Electric Range,Oldest Year,Number of Cars
str,str,f64,i64,u32
"""WA""","""TESLA""",56.715893,2018,85408
"""WA""","""CHEVROLET""",88.73235,2018,8881
"""WA""","""NISSAN""",67.008487,2018,7423
"""WA""","""FORD""",0.083241,2018,7208
"""WA""","""KIA""",35.681039,2018,6239
…,…,…,…,…
"""MD""","""TESLA""",33.733333,2018,15
"""TX""","""TESLA""",105.785714,2018,14
"""NC""","""TESLA""",16.538462,2018,13
